In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn_pandas import CategoricalImputer

ModuleNotFoundError: No module named 'sklearn_pandas'

In [ ]:
data = pd.read_csv("census.csv")

In [ ]:
# numerical
num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loss', 'hours-per-week']

# categorical
cat_cols = ['workclass', 'education_level', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']

# need log transform
log_transform_cols = ['capital-loss', 'capital-gain']

In [ ]:
# select the categorical columsn
def get_cat_cols(X):
    return X[cat_cols]

# select the numerical columns
def get_num_cols(X):
    return X[num_cols]

# select the columns that need log transform
def get_log_transform_cols(X):
    return X[log_transform_cols]

# one-hot encode the categorical variables, replacing nan with 0
def get_dummies(X):
    return pd.get_dummies(X)

# imputer for empty values in categorical variables.
# note: this is not optimal since we are not using the strategy from train in the test
# sample. Not sure how to accomplish that.
def cat_imputer(X):
    return X.apply(lambda col: CategoricalImputer().fit_transform(col)) 

In [ ]:
# log transform
log_transform_pipeline = Pipeline([
 ('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
 ('imputer', SimpleImputer(strategy='mean')),   
 ('log_transform', FunctionTransformer(np.log1p))
])

# for all the numerical cols fill null values with the mean of the column
# and then apply scaling
num_cols_pipeline = Pipeline([
 ('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
 ('imputer', SimpleImputer(strategy='mean')),
 ('min_max_scaler', MinMaxScaler())
])

# for all the categorical cols, apply the categorical imputer function
# from the sklearn_pandas library and then one-hot encode using the pandas
# get_dummies function
cat_cols_pipeline = Pipeline([
 ('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
 ('imputer', FunctionTransformer(cat_imputer, validate=False)),
 ('get_dummies', FunctionTransformer(get_dummies, validate=False))
])

In [ ]:
steps_ = FeatureUnion([
    ('log_transform', log_transform_pipeline),
    ('num_cols', num_cols_pipeline),
    ('cat_cols', cat_cols_pipeline)
])

# this full pipeline will apply the 3 previous steps
full_pipeline = Pipeline([('steps_', steps_)])

In [ ]:
# binarize the target variable
y = data['income'].map({'<=50K': 0, '>50K': 1})

# transform the entire training set.
# this pipeline will be fitted to the training set
# and the test set (for submission) only need to be transformed (not fitted)
X = full_pipeline.fit_transform(data)

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [24]:
clf = AdaBoostClassifier(n_estimators=400, learning_rate=1, random_state=23).fit(X_train, y_train)

In [11]:
probs_train = clf.predict_proba(X_train)[:, 1]
probs_test = clf.predict_proba(X_test)[:, 1]
print("score train: {}".format(roc_auc_score(y_train, probs_train)))
print("score test: {}".format(roc_auc_score(y_test, probs_test)))

score train: 0.9260630332436355
score test: 0.9271893639434359


In [16]:
test = pd.read_csv("test_census.csv")
print("shape1: {}".format(test.shape))

# use the pipeline to transform
X_sub = full_pipeline.transform(test)

# rename the first column to id
test['id'] = test.iloc[:,0] 
print("shape1: {}".format(X_sub.shape))
# make predictions
test['income'] = clf.predict_proba(X_sub)[:, 1]

# generate output file
test[['id', 'income']].to_csv("submission.csv", index=False)

shape1: (45222, 14)
shape1: (45222, 105)
